# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 11:41AM,244375,19,N17727,"NAPP Technologies, LLC",Released
1,Aug 17 2022 11:40AM,244374,22,624548,"NBTY Global, Inc.",Released
2,Aug 17 2022 11:27AM,244330,20,ALUR261204178,Alumier Labs Inc.,Released
3,Aug 17 2022 11:27AM,244330,20,ALUR562256437,Alumier Labs Inc.,Released
4,Aug 17 2022 11:27AM,244330,20,ALUR620750958,Alumier Labs Inc.,Released
5,Aug 17 2022 11:27AM,244330,20,ALUR807702302,Alumier Labs Inc.,Released
6,Aug 17 2022 11:27AM,244330,20,ALUR952433724,Alumier Labs Inc.,Released
7,Aug 17 2022 11:27AM,244330,20,ALU0007558,Alumier Labs Inc.,Released
8,Aug 17 2022 11:27AM,244330,20,ALUR257727459,Alumier Labs Inc.,Released
9,Aug 17 2022 11:27AM,244330,20,ALUR734982065,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,244370,Released,1
49,244372,Released,3
50,244373,Released,4
51,244374,Released,1
52,244375,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244370,NaN,NaN,1.0
244372,NaN,NaN,3.0
244373,NaN,NaN,4.0
244374,NaN,NaN,1.0
244375,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244268,0.0,1.0,0.0
244274,0.0,0.0,1.0
244275,0.0,0.0,1.0
244300,0.0,6.0,2.0
244303,0.0,1.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244268,0,1,0
244274,0,0,1
244275,0,0,1
244300,0,6,2
244303,0,1,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244268,0,1,0
1,244274,0,0,1
2,244275,0,0,1
3,244300,0,6,2
4,244303,0,1,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244268,,1,
1,244274,,,1
2,244275,,,1
3,244300,,6,2
4,244303,,1,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC"
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc."
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.
22,Aug 17 2022 11:26AM,244361,10,Bio-PRF
35,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health
39,Aug 17 2022 11:21AM,244372,10,"Nextsource Biotechnology, LLC"
42,Aug 17 2022 11:03AM,244370,10,Cosmetic Solutions LLC
43,Aug 17 2022 11:03AM,244369,10,"Citieffe, Inc."
44,Aug 17 2022 10:51AM,244367,21,"NBTY Global, Inc."
45,Aug 17 2022 10:50AM,244366,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC",,,1
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc.",,,1
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.,,,20
3,Aug 17 2022 11:26AM,244361,10,Bio-PRF,,,13
4,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health,,,4
5,Aug 17 2022 11:21AM,244372,10,"Nextsource Biotechnology, LLC",,,3
6,Aug 17 2022 11:03AM,244370,10,Cosmetic Solutions LLC,,,1
7,Aug 17 2022 11:03AM,244369,10,"Citieffe, Inc.",,,1
8,Aug 17 2022 10:51AM,244367,21,"NBTY Global, Inc.",,,1
9,Aug 17 2022 10:50AM,244366,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC",1,,
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc.",1,,
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.,20,,
3,Aug 17 2022 11:26AM,244361,10,Bio-PRF,13,,
4,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health,4,,
5,Aug 17 2022 11:21AM,244372,10,"Nextsource Biotechnology, LLC",3,,
6,Aug 17 2022 11:03AM,244370,10,Cosmetic Solutions LLC,1,,
7,Aug 17 2022 11:03AM,244369,10,"Citieffe, Inc.",1,,
8,Aug 17 2022 10:51AM,244367,21,"NBTY Global, Inc.",1,,
9,Aug 17 2022 10:50AM,244366,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC",1,,
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc.",1,,
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.,20,,
3,Aug 17 2022 11:26AM,244361,10,Bio-PRF,13,,
4,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.,20.0,NaN,NaN
3,Aug 17 2022 11:26AM,244361,10,Bio-PRF,13.0,NaN,NaN
4,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 11:41AM,244375,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,Aug 17 2022 11:40AM,244374,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Aug 17 2022 11:27AM,244330,20,Alumier Labs Inc.,20.0,0.0,0.0
3,Aug 17 2022 11:26AM,244361,10,Bio-PRF,13.0,0.0,0.0
4,Aug 17 2022 11:23AM,244373,10,Ivaoes Animal Health,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5864140,105.0,8.0,0.0
102,244320,1.0,0.0,0.0
15,977430,40.0,14.0,0.0
16,488549,2.0,0.0,0.0
17,488643,2.0,0.0,0.0
19,488703,9.0,13.0,20.0
20,977256,45.0,1.0,0.0
21,1710461,7.0,0.0,0.0
22,244374,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5864140,105.0,8.0,0.0
1,102,244320,1.0,0.0,0.0
2,15,977430,40.0,14.0,0.0
3,16,488549,2.0,0.0,0.0
4,17,488643,2.0,0.0,0.0
5,19,488703,9.0,13.0,20.0
6,20,977256,45.0,1.0,0.0
7,21,1710461,7.0,0.0,0.0
8,22,244374,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,8.0,0.0
1,102,1.0,0.0,0.0
2,15,40.0,14.0,0.0
3,16,2.0,0.0,0.0
4,17,2.0,0.0,0.0
5,19,9.0,13.0,20.0
6,20,45.0,1.0,0.0
7,21,7.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,102,Released,1.0
2,15,Released,40.0
3,16,Released,2.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0
Executing,8.0,0.0,14.0,0.0,0.0,13.0,1.0,0.0,0.0
Released,105.0,1.0,40.0,2.0,2.0,9.0,45.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0
1,Executing,8.0,0.0,14.0,0.0,0.0,13.0,1.0,0.0,0.0
2,Released,105.0,1.0,40.0,2.0,2.0,9.0,45.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0
1,Executing,8.0,0.0,14.0,0.0,0.0,13.0,1.0,0.0,0.0
2,Released,105.0,1.0,40.0,2.0,2.0,9.0,45.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()